In [15]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics.cluster import contingency_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import jaccard_score
from scipy.spatial import distance
from sklearn.metrics import normalized_mutual_info_score
from sklearn.metrics import f1_score
import os
import glob
import pandas as pd
import numpy as np



Myfiles = {}
filname = {}
path = 'D:/7thsem/btp/final/cleaned/'
dir_list = os.listdir(path)
            


In [16]:
def BOW(data):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(data)
    unique_words = vectorizer.get_feature_names_out()
    return X,unique_words



In [17]:
def tfidF(data):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(data)
    unique_words = vectorizer.get_feature_names_out()
    return X,unique_words


In [18]:
def lsa(data):
    vectorizer = TruncatedSVD(20, algorithm = 'arpack')
    X = vectorizer.fit_transform(data)
    X = Normalizer(copy=False).fit_transform(X)
    return X,vectorizer.components_,vectorizer.singular_values_


In [19]:
def lda(data):
    vectorizer = LatentDirichletAllocation(n_components=20, random_state=0)
    X = vectorizer.fit_transform(data)
    X = Normalizer(copy=False).fit_transform(X)
    return X,vectorizer.components_

In [20]:
BOW_features = {}
BOW_uniq_words = {}
tfidf_features = {}
tfidf_uniq_words = {}


LSA_Tfeatures = {}
LSA_Tword_topic = {}
LSA_singular = {}
LDA_Tfeatures = {}
LDA_Tword_topic = {}

clusterings_data = []
clusteringd_data = []
clusteringb_label = []
clusteringt_label = []
clusterings_label = []
clusteringd_label = []

In [21]:

def tbfeature():
    cnt = 0
    cluter_docs = []
    for fol in dir_list:
        cnt = 0
        documents = []
        for i in Myfiles[fol]:
            with open(i,'r') as fin:
                data = fin.read()
                documents.append(data)
                cluter_docs.append(data)
                clusteringb_label.append(cnt)
                clusteringt_label.append(cnt)
                clusterings_label.append(cnt)
                clusteringd_label.append(cnt)
        BOW_features[fol],BOW_uniq_words[fol] = BOW(documents)
        tfidf_features[fol],tfidf_uniq_words[fol] = tfidF(documents)
        
    return tfidF(cluter_docs)[0], BOW(cluter_docs)[0]
    
        



In [22]:
def ldfeature():

    for fol in dir_list:
   
        LSA_Tfeatures[fol], LSA_Tword_topic[fol], LSA_singular[fol] = lsa(tfidf_features[fol].toarray())
  
        LDA_Tfeatures[fol], LDA_Tword_topic[fol] = lda(tfidf_features[fol].toarray())

        

In [23]:
def purity_score(y_true, y_pred):
    # compute contingency matrix (also called confusion matrix)
    confusion_matrix = contingency_matrix(y_true, y_pred)
    # return purity
    return np.sum(np.amax(confusion_matrix, axis=0)) / np.sum(confusion_matrix)




In [24]:
def f1_score(true_labels,labels):
    f_measure = f1_score(true_labels, labels, average='macro')
    return f_measure

In [25]:
def NMI(true_labels,labels):
    nmi = normalized_mutual_info_score(true_labels, labels)
    return nmi

In [26]:
def kmeans(data, k, max_iters=100, similarity_measure="cosine", threshold=0.5):
    n, m = data.shape
    centroids = data[np.random.choice(n, k, replace=False)]
    labels = np.zeros(n)

    for _ in range(max_iters):
        if similarity_measure == "cosine":
            similarities = cosine_similarity(data, centroids)
            # if _ == 0:
            #     print(similarities)
            new_labels = np.argmax(similarities, axis=1)
        elif similarity_measure == "jaccard":
            # Binarize the data and centroids
            binarized_data = (data > threshold).astype(int)
            binarized_centroids = (centroids > threshold).astype(int)
            
            jaccard_coeffs = np.zeros((n, k))
            for i in range(n):
                for j in range(k):
                    jaccard_coeffs[i, j] = jaccard_score(binarized_data[i], binarized_centroids[j])
            new_labels = np.argmin(jaccard_coeffs, axis=1)
        elif similarity_measure == "mahalanobis":
            mahalanobis_distances = np.zeros((n, k))
            cov = np.cov(data, rowvar=False)
            # Add a small constant to the diagonal elements of the covariance matrix
            cov += np.eye(cov.shape[0]) * 1e-6
            VI = np.linalg.inv(cov)
            for j in range(k):
                mahalanobis_distances[:, j] = distance.cdist(data, centroids[j].reshape(1, -1), 'mahalanobis', VI=VI)[:, 0]
            new_labels = np.argmin(mahalanobis_distances, axis=1)
        elif similarity_measure == "euclidean":
            euclidean_distances = np.linalg.norm(data[:, np.newaxis] - centroids, axis=2)
            new_labels = np.argmin(euclidean_distances, axis=1)
        else:
            raise ValueError("Invalid similarity_measure. Choose 'cosine', 'jaccard', 'mahalanobis', or 'euclidean'.")
        
        if np.array_equal(labels, new_labels):
            break
        
        labels = new_labels
        
        for i in range(k):
            cluster_points = data[labels == i]
            if len(cluster_points) > 0:
                centroids[i] = cluster_points.mean(axis=0)
    
    return labels, centroids


In [27]:
# define dictionary for distance measures
distance_measures = ["cosine","jaccard","mahalanobis","euclidean"]

# function defined to compute purity score using pyclustering for various distance measures
def pyPurity(dist_measure,X,y,k):
    pyLabels, pycentroids = kmeans(X, k,100, dist_measure)
    # function purity score is defined in previous section
    return purity_score(y, pyLabels)

# print results


In [34]:
import seaborn as sns
import matplotlib.pylab as plt
# print(Myfiles['alt.atheism'])

# print(len(bow_dict[dir_list[]]["49960.txt"]))
# # print(Feature_word['alt.atheism'])
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)

df = pd.DataFrame(LSA_Tfeatures['alt.atheism'], index=filname['alt.atheism'])
print(df)
# print(df[["god", "moral", "one", "say", "would", "believ", "atheist", "think"]].head(10))

# print(pd.DataFrame(LSA_Tword_topic['alt.atheism'], columns=BOW_uniq_words['alt.atheism']).head(10))


# print(pd.DataFrame(BOW_uniq_words['alt.atheism']))
# topic_term = pd.DataFrame(np.transpose(LDA_Tword_topic['alt.atheism']), index=BOW_uniq_words['alt.atheism'])
# print(topic_term)
# data = topic_term[0]
# data = data.sort_values(ascending=False)
# top_10 = data[:10]
# plt.title('Top terms along the axis of Latent concept 1')
# fig = sns.barplot(x= top_10.values, y=top_10.index)
# sns.barplot(x=list(range(len(LSA_Tfeatures['alt.atheism']))), y = LSA_Tfeatures['alt.atheism'][0])
# print(df)

            


                 0         1         2         3         4         5         6         7         8         9         10        11        12        13        14        15        16        17        18        19
49960.txt  0.729965 -0.226510 -0.258614 -0.016436  0.034608 -0.062201 -0.220535 -0.150730  0.151113 -0.147095  0.232441 -0.019620 -0.003997  0.081559  0.190842  0.084144  0.121150  0.284212 -0.132459  0.101080
51060.txt  0.832851 -0.101530 -0.284992  0.206116  0.325032 -0.026258 -0.173386  0.097267  0.047885 -0.052171 -0.009816 -0.037766 -0.046234  0.078231  0.024131  0.040930  0.075921  0.056172  0.015103  0.016968
51119.txt  0.591391 -0.112526 -0.065543 -0.254149 -0.207306 -0.156678  0.111163 -0.314215  0.107289 -0.338774 -0.051795 -0.039182  0.080862  0.172509 -0.252948  0.007038 -0.355925  0.158437 -0.036179 -0.078582
51120.txt  0.663296 -0.199161 -0.076615 -0.276073 -0.055336 -0.239950 -0.285051 -0.163485  0.123790 -0.167192  0.031262  0.126009 -0.187301  0.067623 -0.236563 

In [29]:





for fol in dir_list:
    Myfiles[fol] = [I for I in glob.glob(path+fol+"/*.txt")]

for fol in dir_list:
    filname[fol] = []
    for i in Myfiles[fol]:
        name = i.split('/')
        name = name[len(name)-1].split('\\')
        filname[fol].append(name[1])
        

clusteringt_data,clusteringb_data = tbfeature()
ldfeature()


In [30]:

# print(clusteringb_data.toarray())

In [31]:
# import warnings
# warnings.filterwarnings('ignore')
# for measure in distance_measures:
    
    # print(f"The purity score for {measure} distance is {round(pyPurity(measure,clusteringt_data.toarray(),clusteringt_label,20)*100, 3)}%")
